In [49]:
from requests import post
from requests.exceptions import JSONDecodeError
import uuid
import os
from dotenv import load_dotenv

# Для ipynb явно загружаются переменные окружения
load_dotenv('.env')

GIGACHAT_AUTH_URL = os.environ.get("GIGACHAT_AUTH_URL")
GIGACHAT_AUTHORIZE_KEY = os.environ.get("GIGACHAT_AUTHORIZE_KEY")

if not GIGACHAT_AUTH_URL:
    raise ValueError("GIGACHAT_AUTH_URL не установлен в окружении")

if not GIGACHAT_AUTHORIZE_KEY:
    raise ValueError("GIGACHAT_AUTHORIZE_KEY не установлен в окружении")

# Обновлять токен доступа, если до окончания действия осталось 60 секунд
REFRESH_ACCESS_TOKEN_BEFORE_S = 60
__access_token = ""
__access_token_expires = ""

def refresh_access_token():
    """Refresh access_token for llm and set access_token_expires time

    Returns:
        Boolean: 
            True if refresh is success
            False if token is not refreshed
    """

    global __access_token
    global __access_token_expires

    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Accept": "application/json",
        "RqUID": f"{uuid.uuid4()}",
        "Authorization": f"Basic {GIGACHAT_AUTHORIZE_KEY}"
    }

    payload = {
        "scope": "GIGACHAT_API_PERS"
    }

    try:
        response = post(
            url=GIGACHAT_AUTH_URL,
            headers=headers,
            data=payload
        )

        response_json = response.json()

        __access_token = response_json.get("access_token")
        __access_token_expires = response_json.get("expires_at")

        if __access_token is None or __access_token_expires is None:
            print("Ошибка при выполнении запроса на обновление токена доступа",
                  "\nВ полученных данных отсутствуют 'access_token' или 'expires_at",
                  f"\nОтвет сервиса: {response_json}")
            return False
    except JSONDecodeError as ex:
        print(f"Ошибка при парсинге ответа от сервиса при выполнении запроса на обновление токена доступа\n{ex}")
        return False
    except Exception as ex:
        print(f"Ошибка при выполнении запроса на обновление токена доступа\n{ex}")
        return False
    
    return True


In [50]:
is_refresh_success = refresh_access_token()
print(is_refresh_success)

True


In [44]:
from datetime import datetime

# При обращении к функции передачи сообщения нужна проверка действительности токена и его обновление

def refresh_access_token_if_expire(func):
    def wrapper(*args, **kwargs):
        # Проверка действительности токена
        # Преобразование в секунды для корректной работы с datetime
        token_expire_time_s = __access_token_expires / 1000.0
        token_expire_datetime = datetime.fromtimestamp(token_expire_time_s)
        # Остаток времени до истечения токена в секундах
        time_left_s = (token_expire_datetime - datetime.now()).total_seconds()

        print(
                "Refreshing access token",
                f"Expire datetime is: {token_expire_datetime}",
                f"Seconds left: {time_left_s}"
            )
        
        # Если действие токена скоро закончится или уже закончилось - обновить
        if time_left_s < REFRESH_ACCESS_TOKEN_BEFORE_S:
            print(
                "Refreshing access token",
                f"Expire datetime is: {token_expire_datetime}",
                f"Seconds left: {time_left_s}"
            )
            refresh_access_token()
        
        return func(*args, **kwargs)

    return wrapper

In [45]:
# Нужна выгрузка истории сообщений по достижении определенного лимита, чтобы не хранить всё в оперативной памяти
# Нужно лимитировать историю сообщений, передаваемую LLM

# Формат хранения истории сообщений
# message_history = [
#     (
#         {
#             "author": "user",
#             "message": "request text"
#         },
#         {
#             "author": "agent",
#             "message": "answer text"
#         }
#     )
# ]

message_history = []

@refresh_access_token_if_expire
def bot_make_request(request_message: str):
    pass

In [46]:
bot_make_request("message")

Refreshing access token Expire datetime is: 2026-02-14 22:54:46.875000 Seconds left: 1731.684362
